# PTST Idealization using axi-symmetric 2D interface layer

## Purpose of the notebook

Verify the axi-symmetric model in an elastic state to ensure that the confinement is
correctly reproduced. This notebook serves as a basis for 3D degradation within
a ligament/bond zone in axi-symmetric models. Examples are
 - PTST
 - prestressing
 - confined bond configuration
 - effect of splitting cracks
 - restrained splitting cracks in fatigue tests

## Implementation remarks

- The notebook now uses `BCSliceI` - to ensure the same order of nodes along an 
interfaces
- Compression is applied using individual `BCDof` instances without integration 
  over the boundary. This should be changed to BCSliceE - which can use the 
  finite element shape functions to integrate over the element domain

In [117]:
%matplotlib widget
import time
from bmcs_expsim.utils.mlab_decorators import decorate_figure
from mayavi import mlab
import numpy as np
np.seterr(divide='ignore', invalid='ignore') 
import warnings
import matplotlib.pylab as plt
import numpy as np
from ibvpy.api import TStepBC, TFCyclicNonsymmetricConstant, TFBilinear
from ibvpy.bcond import BCSliceI, BCDof
from ibvpy.xmodel.xdomain_fe_grid_axisym import XDomainFEGridAxiSym
from ibvpy.fets import FETS2D4Q
from ibvpy.tmodel.viz3d_scalar_field import \
    Vis3DStateField, Viz3DScalarField
from ibvpy.tmodel.viz3d_tensor_field import \
    Vis3DTensorField, Viz3DTensorField
from ibvpy.tmodel.mats3D.mats3D_elastic.vmats3D_elastic import \
    MATS3DElastic
from ibvpy.tmodel.mats2D.mats2D_elastic.vmats2D_elastic import \
    MATS2DElastic

## FE Axisymmetric discretization

In [119]:
n_x_e = 20
n_inner_y_e = 10
n_outer_y_e = 10
L_x = 100.0 # [mm]
R_in = 25 # [mm]
dR = 10 # [mm]
R_out = 50 # [mm]
xd_inner = XDomainFEGridAxiSym(integ_factor = 2 * np.pi,
                               coord_min=(0, 0),
                               coord_max=(L_x, R_in),
                               shape=(n_x_e, n_inner_y_e),
                               fets=FETS2D4Q())
xd_middle = XDomainFEGridAxiSym(integ_factor = 2 * np.pi,
                               coord_min=(0, R_in),
                               coord_max=(L_x, R_in+dR),
                               shape=(n_x_e, 1),
                               fets=FETS2D4Q())
xd_outer = XDomainFEGridAxiSym(integ_factor = 2 * np.pi,
                               coord_min=(0, R_in+dR),
                               coord_max=(L_x, R_out),
                               shape=(n_x_e, n_outer_y_e),
                               fets=FETS2D4Q())
m_elastic = MATS3DElastic(E=37000, nu=0.18)
m_inelastic = MATS3DElastic(E=17000, nu=0.18)
m = TStepBC(
    domains=[(xd_inner, m_elastic),
             (xd_outer, m_elastic),
             (xd_middle, m_inelastic),
             ]
)

## Boundary and transition conditions

### Kinematic links between the domains

In [120]:
link_inner_middle = BCSliceI(var='u', dims=[0, 1],
                             slice=xd_inner.mesh.I[:, -1], 
                             link_slice=xd_middle.mesh.I[:, 0],
                             link_coeffs=[1.0, 1.0],
                             link_dims=[0, 1],
                             value=0.0
                            )
link_middle_outer = BCSliceI(slice=xd_outer.mesh.I[:, 0], 
                             var='u', dims=[0, 1],
                             link_slice=xd_middle.mesh.I[:, -1],
                             link_coeffs=[1, 1],
                             link_dims=[0, 1],
                             value=0,
                            )

### Compression/Traction fc

In [121]:
inner_fixed_1 = BCSliceI(slice=xd_inner.mesh.I[:, 0], var='u', dims=[1], value=0)
outer_fixed_0 = BCSliceI(slice=xd_outer.mesh.I[0, :], var='u', dims=[0], value=0)

outer_compression_slice = BCSliceI(slice=xd_outer.mesh.I[:, -1],
                                   var='u', dims=[1], value=0.01)

compression_dofs = outer_compression_slice.dofs
compression = -10000*L_x / len(compression_dofs)
outer_compression_force_first = [BCDof(var='f', dof=dof, value=compression) 
                                 for dof in compression_dofs ]
outer_compression_force_first[0].value *=0.5
outer_compression_force_first[-1].value *=0.5
bc1 = [inner_fixed_1, 
       outer_fixed_0,
       link_inner_middle,
       link_middle_outer
      ] + outer_compression_force_first

### Sliding dc

In [122]:
#lower_fixed_0 = BCSlice(slice=xd_lower.mesh[:, 0, :, 0], var='u', dims=[1], value=0)
#upper_fixed_1 = BCSlice(slice=xd_upper.mesh[0, :, 0, :], var='u', dims=[0], value=0)
#
#lower_slide = BCSlice(slice=xd_lower.mesh[0, :, 0, :], var='u', dims=[0], value=6)
#
#
#bc1 =   [lower_fixed_0, upper_fixed_1, lower_slide]

### Sliding fc

In [123]:
#lower_fixed_0 = BCSlice(slice=xd_lower.mesh[:, 0, :, 0], var='u', dims=[1], value=0)
#upper_fixed_1 = BCSlice(slice=xd_upper.mesh[0, :, 0, :], var='u', dims=[0], value=0)
#
#
#
#lower_slide = BCSlice(slice=xd_lower.mesh[0, :, 0, :], var='u', dims=[0], value=0)
#Force = 10 / len(lower_slide.dofs)
#lower_slide_force = [BCDof(var='f', dof=dof, value = Force) 
#             for dof in lower_slide.dofs]
#
#bc1 =   [lower_fixed_0, upper_fixed_1] + lower_slide_force

In [124]:
m.bc=bc1
m.hist.vis_record = {
#    'strain': Vis3DTensorField(var='eps_ab'),
    'stress': Vis3DTensorField(var='sig_ab'),
    #        'kinematic hardening': Vis3DStateField(var='z_a')
}

s = m.sim
s.tloop.verbose = True
s.tloop.k_max = 1000
s.tline.step = 0.1
# Accessing the following property - ensures serialization
s.tstep.fe_domain.serialized_subdomains

In [125]:
s.reset()
s.run()

t:	 0.00(0), 
	 0.10(1), 
	 0.20(1), 
	 0.30(1), 
	 0.40(1), 
	 0.50(1), 
	 0.60(1), 
	 0.70(1), 
	 0.80(1), 
	 0.90(1), 
	 1.00(1), 



## Postprocesing

### Access the last displacement step

In [126]:
U_last = m.hist.U_t[-1]

In [127]:
U_last[xd_middle.o_Ia], U_last[xd_inner.o_Ia]

(array([[-0.00060515, -0.01756049],
        [-0.00156554, -0.03183589],
        [ 0.00124372, -0.01716248],
        [ 0.00123678, -0.0318031 ],
        [ 0.00316061, -0.01713656],
        [ 0.00314531, -0.03183824],
        [ 0.0049695 , -0.01715343],
        [ 0.00492394, -0.03181516],
        [ 0.00671735, -0.01717145],
        [ 0.0066658 , -0.03179951],
        [ 0.00843566, -0.01718646],
        [ 0.00838787, -0.03178618],
        [ 0.0101385 , -0.01719709],
        [ 0.01009797, -0.03177511],
        [ 0.01183325, -0.01720409],
        [ 0.01180059, -0.03176681],
        [ 0.01352397, -0.01720867],
        [ 0.01349878, -0.03176182],
        [ 0.01521299, -0.01721194],
        [ 0.01519457, -0.03176036],
        [ 0.01690172, -0.01721456],
        [ 0.01688939, -0.03176249],
        [ 0.01859124, -0.01721662],
        [ 0.01858441, -0.03176824],
        [ 0.02028267, -0.01721758],
        [ 0.02028101, -0.03177759],
        [ 0.02197755, -0.01721618],
        [ 0.02198132, -0.031

In [128]:
xd_middle.map_U_to_field(U_last)

array([[[[ 4.10071924e-04, -5.86156473e-06,  0.00000000e+00],
         [-5.86156473e-06, -1.43525834e-03,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00, -7.56436085e-04]],

        [[ 4.10071924e-04,  2.16618912e-05,  0.00000000e+00],
         [ 2.16618912e-05, -1.45634454e-03,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00, -7.49604234e-04]],

        [[ 5.20165748e-04,  5.75691970e-07,  0.00000000e+00],
         [ 5.75691970e-07, -1.45634454e-03,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00, -8.73677513e-04]],

        [[ 5.20165748e-04, -2.69477640e-05,  0.00000000e+00],
         [-2.69477640e-05, -1.43525834e-03,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00, -8.75608152e-04]]],


       [[[ 3.83024318e-04,  8.66023775e-07,  0.00000000e+00],
         [ 8.66023775e-07, -1.46535291e-03,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00, -7.47002158e-04]],

        [[ 3.83024318e-04,  6.24808497e-07,  0.00000000e+0

In [129]:
if True:
    mlab.options.backend = 'envisage'
    mlab.options.offscreen = False # 'envisage'
    f_strain = mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = 'stress'
    strain_viz = Viz3DTensorField(vis3d=m.hist['stress'])
    strain_viz.setup()
    strain_viz.warp_vector.filter.scale_factor = 0.1
    strain_viz.plot(s.tstep.t_n)
    mlab.show()